In [ ]:
import keras
import tensorflow as tf 
import numpy as np
import util_mnist_reader
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,classification_report

#Loading Dataset                            
X_train, y_train = util_mnist_reader.load_mnist('data/fashion', kind='train')
X_test, y_test = util_mnist_reader.load_mnist('data/fashion', kind='t10k')

labelNames =  np.array(['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'])
labels=10

#Normalizing and Reshaping
X_train = X_train/255
X_test = X_test/255

## One Hidden Layer Neural Network 

In [ ]:
examples_train = y_train.shape[0]
examples_test = y_test.shape[0]

#Reshaping
y_train_reshape = y_train.reshape(1, examples_train)
y_test_reshape = y_test.reshape(1, examples_test)

y_train_NN= np.eye(labels)[y_train_reshape.astype('int32')]
y_train_NN=y_train_NN.astype('int32')
y_train_NN = y_train_NN.T.reshape(labels, examples_train)

y_test_NN = np.eye(labels)[y_test_reshape.astype('int32')]
y_test_NN=y_test_NN.astype('int32')
y_test_NN = y_test_NN.T.reshape(labels, examples_test)

#Shuffling
m1 = 60000
m2 = 10000

X_train = X_train[:m1].T
X_test = X_test[:m2].T

shuffle_index = np.random.permutation(m1)
X_train, y_train_NN = X_train[:, shuffle_index], y_train_NN[:, shuffle_index]
shuffle_index = np.random.permutation(m2)
X_test, y_test_NN = X_test[:, shuffle_index], y_test_NN[:, shuffle_index]

#Sigmoid Function
def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s

#Loss Function
def compute_loss(Y, Y_hat):

    L_sum = np.sum(np.multiply(Y, np.log(Y_hat)))
    m = Y.shape[1]
    L = -(1/m) * L_sum

    return L

n_x = X_train.shape[0]
n_h = 64
learning_rate = 0.5

W1 = np.random.randn(n_h, n_x)
b1 = np.zeros((n_h, 1))
W2 = np.random.randn(labels, n_h)
b2 = np.zeros((labels, 1))

losstrack=[]
for i in range(2000):

    Z1 = np.matmul(W1,X_train) + b1
    A1 = sigmoid(Z1)
    Z2 = np.matmul(W2,A1) + b2
    A2 = np.exp(Z2) / np.sum(np.exp(Z2), axis=0)

    cost = compute_loss(y_train_NN, A2)
    losstrack.append(cost)

    dZ2 = A2-y_train_NN
    dW2 = (1./m1) * np.matmul(dZ2, A1.T)
    db2 = (1./m1) * np.sum(dZ2, axis=1, keepdims=True)

    dS1 = np.matmul(W2.T, dZ2)
    dZ1 = dS1 * sigmoid(Z1) * (1 - sigmoid(Z1))
    dW1 = (1./m1) * np.matmul(dZ1, X_train.T)
    db1 = (1./m1) * np.sum(dZ1, axis=1, keepdims=True)

    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1

    if (i % 100 == 0):
        print("Cost of Epoch", i, ": ", cost)

print("Final cost:", cost)

#Confusion-matrix and Classification Report
Z1 = np.matmul(W1, X_test) + b1
A1 = sigmoid(Z1)
Z2 = np.matmul(W2, A1) + b2
A2 = np.exp(Z2) / np.sum(np.exp(Z2), axis=0)


predictions = np.argmax(A2, axis=0)

print(confusion_matrix(predictions, np.argmax(y_test_NN, axis=0)))
print(classification_report(predictions, np.argmax(y_test_NN, axis=0)))


#Plotting Loss and Accuracy Graph
plt.plot(losstrack)
plt.title('Cost VS Iterations')
plt.xlabel('Number of Iterations')
plt.ylabel('Cost')

## Multi-layer Convolution Network

In [ ]:
X_train = X_train.reshape([-1, 28, 28, 1])
X_test = X_test.reshape([-1, 28, 28, 1])

# Modelling 3-layer neural network
model_3 = keras.Sequential([
    keras.layers.Flatten(input_shape=[28,28,1]),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
    
model_3.summary() 

# Compile the model
model_3.compile(optimizer=keras.optimizers.Adam(1e-4),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

#Train the NN-3 with 10 epochs
N1 = 10  
A1=model_3.fit(X_train, y_train, epochs=N1, validation_split=0.1)

#Evaluate the model
test_loss, test_acc = model_3.evaluate(X_test, y_test)
print("Model - 3 layers - test loss:", test_loss * 100)
print("Model - 3 layers - test accuracy:", test_acc * 100)

#Confusion-Matrix and Classification Report
predictions_3=model_3.predict(X_test)
predictions_3=np.argmax(predictions_3, axis=1)

print(confusion_matrix(predictions_3, keras.utils.np_utils.to_categorical(y_test).argmax(axis=1)))
print(classification_report(predictions_3, keras.utils.np_utils.to_categorical(y_test).argmax(axis=1)))

#Plotting Loss and Accuracy Graph
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N1), A1.history["loss"], label="train_loss")
plt.plot(np.arange(0, N1), A1.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N1), A1.history["acc"], label="train_acc")
plt.plot(np.arange(0, N1), A1.history["val_acc"], label="val_acc")
plt.title("Loss and Accuracy")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")


# Modelling 6-layer neural network 
model_6 = keras.Sequential([
    keras.layers.Flatten(input_shape=[28,28,1]),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
    
model_6.summary() 

model_6.compile(optimizer=keras.optimizers.Adam(1e-4),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

#Train the NN-6 with 10 epochs 
N2 = 10
A2=model_6.fit(X_train, y_train, epochs=N2, validation_split=0.1)

#Evaluate the model
test_loss, test_acc = model_6.evaluate(X_test, y_test)
print("Model - 6 layers - test loss:", test_loss * 100)
print("Model - 6 layers - test accuracy:", test_acc * 100)

#Confusion-Matrix and Classification Report
predictions_6=model_6.predict(X_test)
predictions_6=np.argmax(predictions_6, axis=1)

print(confusion_matrix(predictions_6, keras.utils.np_utils.to_categorical(y_test).argmax(axis=1)))
print(classification_report(predictions_6, keras.utils.np_utils.to_categorical(y_test).argmax(axis=1)))

#Plotting Loss and Accuracy Graph
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N2), A2.history["loss"], label="train_loss")
plt.plot(np.arange(0, N2), A2.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N2), A2.history["acc"], label="train_acc")
plt.plot(np.arange(0, N2), A2.history["val_acc"], label="val_acc")
plt.title("Loss and Accuracy")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")



# Modelling 12-layer neural network 
model_12 = keras.Sequential([
    keras.layers.Flatten(input_shape=[28,28,1]),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
    
model_12.summary()
 
model_12.compile(optimizer=keras.optimizers.Adam(1e-4),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

#Train the NN-12 with 10 epochs
N3 = 10
A3=model_12.fit(X_train, y_train, epochs=N3, validation_split=0.1)

#Evaluate the model
test_loss, test_acc = model_12.evaluate(X_test, y_test)
print("Model - 12 layers - Test loss:", test_loss * 100)
print("Model - 12 layers - Test accuracy:", test_acc * 100)

#Confusion-Matrix and Classification Report
predictions_12=model_12.predict(X_test)
predictions_12=np.argmax(predictions_12, axis=1)

print(confusion_matrix(predictions_12, keras.utils.np_utils.to_categorical(y_test).argmax(axis=1)))
print(classification_report(predictions_12, keras.utils.np_utils.to_categorical(y_test).argmax(axis=1)))

#Plotting Loss and Accuracy Graph
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N3), A3.history["loss"], label="train_loss")
plt.plot(np.arange(0, N3), A3.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N3), A3.history["acc"], label="train_acc")
plt.plot(np.arange(0, N3), A3.history["val_acc"], label="val_acc")
plt.title("Loss and Accuracy")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")

## Convolution Neural Network

In [ ]:
#One-Hot Encoding
y_train_CNN = keras.utils.np_utils.to_categorical(y_train)
y_test_CNN = keras.utils.np_utils.to_categorical(y_test)

#Keras Convolution Model
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (5, 5), padding="same", input_shape=[28, 28, 1]),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(64, (5, 5), padding="same"),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])
    
model.summary()

N_epochs=20
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(1e-4),metrics=['accuracy'])

A=model.fit(X_train, y_train_CNN, validation_split=0.10, batch_size=128, epochs=N_epochs, verbose=2)

#Calculating Test-Loss and Accuracy
test_loss, test_accuracy = model.evaluate(X_test, y_test_CNN, verbose=0)
print("Test loss: ", test_loss*100)
print("Test accuracy: ",test_accuracy*100)

#Confusion-Matrix and Classification Report
predictions_cnn=model.predict(X_test)
predictions_cnn=np.argmax(predictions_cnn, axis=1)

print(confusion_matrix(predictions_cnn, y_test_CNN.argmax(axis=1)))
print(classification_report(predictions_cnn, y_test_CNN.argmax(axis=1)))


#Plotting Loss and Accuracy Graph
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N_epochs), A.history["loss"], label="train_loss")
plt.plot(np.arange(0, N_epochs), A.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N_epochs), A.history["acc"], label="train_acc")
plt.plot(np.arange(0, N_epochs), A.history["val_acc"], label="val_acc")
plt.title("Loss and Accuracy")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")